# Measuring the ATC Induction Profile of mNeon Green

In [16]:
import numpy as np
import pandas as pd
import bokeh.plotting
import glob
import bokeh.io
import bokeh.image
import skimage.io
bokeh.io.output_notebook()

ModuleNotFoundError: No module named 'bokeh.image'

This notebook performs a cursory analysis of the ATC induction profile of our newly-minted mNeonGreen tagged LacI dimers. We ultimately wish to find the appropriate induction profile that will permit observation of single puncta without a high background. This may result in only a few cells per field of view with appreciable fluorescent signal.


## Testing Segmentation

Muir (@muirjm) took a collection of images of cells with the mNeonGreen-LacI fusion expressed in a background which had a 5xOid array integrated at the *yffO* locus. He took images in both phase contrast and mNeon. We will first load the data and determine a segmentation scheme that seems appropriate. I have imported a few functions I used in the MscL project for segmentation that may prove useful.

In [17]:
# get a list of all of the files acquired at 1000 ms exposure
folders = glob.glob('../../../data/images/20200115*/*1000ms*')

# Choose the first entry and find the images (excluding metadata fro now). 
image_files = glob.glob(f'{folders[0]}/*.tif')

The images are saved as `ome.tif` files which are multi-dimensional files. There should be one phase contrast and one fluorescence image. Let's look at each

In [21]:
# Load the image
ome_image = skimage.io.imread(image_files[0])
ome_image

NameError: name 'skimage' is not defined

In [23]:
skimage.io.imread(image_files[0])

NameError: name 'skimage' is not defined